In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy
print("Numpy version imported successfully",numpy.__version__)

import transformers
from transformers import BlipProcessor, BlipForConditionalGeneration
print("Transformers version imported successfully", transformers.__version__)

from PIL import Image
from datasets import load_dataset
print("Datasets loaded")

import easyocr
print("EasyOCR version:", easyocr.__version__)
reader = easyocr.Reader(['en'])

Numpy version imported successfully 1.26.0
Transformers version imported successfully 4.40.0
Datasets loaded


EasyOCR version: 1.7.2
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
import pandas
import os
from tqdm import tqdm
from PIL import Image
import torch
import cv2
import json
from sklearn.cluster import DBSCAN
import numpy as np
from transformers import pipeline

In [ ]:
from datasets import load_dataset
import itertools

dataset = load_dataset("VLR-CVC/ComicsPAP", "sequence_filling", split="val", streaming=True)

# OCR + text clsutering + correction :

In [ ]:
def preprocess_image(img):
    img_np=np.array(img)
    img_bgr=cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    return img_bgr



reader = easyocr.Reader(['en'])
def text_ocr(image,reader=easyocr.Reader(['en'])):
    image=preprocess_image(image)
    results = reader.readtext(image)
    return results


## regrouper des paroles d'un même personnage
def organiser_ocr(result):
    if not result:
      print("OCR result is empty.")
      return ""

    # 1. calculer le point central de chaque box text
    coords = []
    for img_result in result :
        box=img_result[0]
        text=img_result[1]


        x=sum([pt[0] for pt in box])/4
        y = sum([pt[1] for pt in box]) / 4
        coords.append([x, y])

    coords = np.array(coords)

    # 2. DBSCAN clustering 
    clustering = DBSCAN(eps=60, min_samples=1).fit(coords)
    labels = clustering.labels_  

    texts=[]
    for cluster_id in set(labels):

        indices = np.where(labels == cluster_id)[0]
        text_group=[result[indice][1] for indice in indices]
        texts.extend(text_group)

    return ' '.join(texts).lower()


In [ ]:
## corriger des erreurs orthographiques:

from spellchecker import SpellChecker
spell = SpellChecker()
def correct_spelling(text):
    words = text.split()
    corrected_words = []
    for word in words:
        correction = spell.correction(word)
        if correction is None:
            corrected_words.append(word)
        else:
            corrected_words.append(correction)
    return " ".join(corrected_words)


## transformer les textes/images en vecteurs via clip

In [ ]:
from transformers import CLIPTokenizer, CLIPTextModel
import torch

clip_tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
clip_text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")

def ocr_text2embedding(text):
    inputs = clip_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = clip_text_encoder (**inputs)
    # 取 [CLS] token 输出，shape: (1, 768)
    return outputs.last_hidden_state[:, 0, :].squeeze(0)


In [ ]:
def image_ocr(image):#image2text
    image=preprocess_image(image)
    result=text_ocr(image)#默认reader
    text=organiser_ocr(result)
    text=correct_spelling(text)
    text_embeddings=ocr_text2embedding(text)
    return text, text_embeddings

In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

@torch.no_grad()
def image2embedding(image):
    inputs = clip_processor(images=image, return_tensors="pt")
    outputs = clip_model.get_image_features(**inputs)
    return outputs[0]  # shape: (512,)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
def combine_embeddings(img_embed, ocr_embed, mode="concat"):
    if mode == "concat":
        return torch.cat([img_embed, ocr_embed], dim=-1)  # shape: (1024,)
    elif mode == "sum":
        return img_embed + ocr_embed  # shape: (512,)
    elif mode == "average":
        return (img_embed + ocr_embed) / 2
    else:
        raise ValueError("Unknown combination mode")

In [ ]:
def safe_panel_embedding(image, i, is_option=False):
    try:
        ocr_text, ocr_embed = image_ocr(image)
        if not ocr_text or not isinstance(ocr_text, str):
            raise ValueError("Invalid OCR result")

        image_embed = image2embedding(image)
        embedding = combine_embeddings(ocr_embed, image_embed)
        if embedding is None:
            raise ValueError("Embedding failed")
        return ocr_text, embedding

    except Exception as e:
        print(f"[Warning] {'option' if is_option else 'context'} OCR/embedding failed at sample {i}: {e}")
        return "[ERROR]", np.zeros((1024,),dtype=np.float32)

dataset_text=[]
dataset_emb=[]
for i,sample in tqdm(enumerate(dataset)):
  index=sample['index']
  solution_index = sample['solution_index']

  context_text=[]
  option_text=[]
  context_embeddings=[]
  option_embeddings=[]

  for image in sample['context']:
    ocr_text, emb=safe_panel_embedding(image, i, is_option=False)
    context_text.append(ocr_text)
    context_embeddings.append(emb)


  for image in sample['options']:
    ocr_text, emb=safe_panel_embedding(image, i, is_option=True)
    option_text.append(ocr_text)
    option_embeddings.append(emb)

  sample_text={
        'context': context_text,
        'options': option_text,
        "index":index,
        'solution_index': solution_index,
    }


  sample_emb={
        'context': [emb.tolist() for emb in context_embeddings],
        'options': [emb.tolist() for emb in option_embeddings],
        "index":index,
        'solution_index':solution_index,
    }
  dataset_text.append(sample_text)
  dataset_emb.append(sample_emb)
  print(f'{i} sample embeded!')


import os

save_dir = "/content/drive/MyDrive/projet_bd/results"
os.makedirs(save_dir, exist_ok=True)

with open("/content/drive/MyDrive/projet_bd/results/dataset_text2.json", 'w') as f:
        json.dump(dataset_text, f, indent=2, ensure_ascii=False)

with open("/content/drive/MyDrive/projet_bd/results/dataset_embeddings2.json", 'w') as f:
        json.dump(dataset_emb, f, indent=2, ensure_ascii=False)



1it [00:16, 16.26s/it]

0 sample embeded!


2it [00:26, 12.89s/it]

1 sample embeded!
OCR result is empty.
[Warning] context OCR/embedding failed at sample 2: Invalid OCR result


3it [00:32,  9.57s/it]

...

259 sample embeded!


261it [1:07:15, 17.67s/it]

260 sample embeded!


262it [1:07:20, 15.42s/it]

261 sample embeded!


In [ ]:
import torch
import json
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

class ComicPAPDataset(Dataset):
  def __init__(self,json_path):
    with open(json_path,'r') as f:
      self.samples=json.load(f)
  def __len__(self):
    return len(self.samples)

  def __getitem__(self,idx):
    sample=self.samples[idx]
    context=[torch.tensor(vec, dtype=torch.float32) for vec in sample['context']]
    context=torch.stack(context)
    options=[torch.tensor(vec, dtype=torch.float32) for vec in sample['options']]
    options=torch.stack(options)
    label=torch.tensor(sample['solution_index'],dtype=torch.long)

    return {
        'context':context,
        'options':options,
        'label':label
        }


In [ ]:
def pad_collate_fn(batch):
    max_opt_len = max([sample['options'].shape[0] for sample in batch])

    context_batch = []
    options_batch = []
    label_batch = []
    option_mask = []
    for sample in batch:
        context_batch.append(sample['context'])  # shape: (num_ctx, 1024)

        options = sample['options']              # shape: (num_opt, 1024)
        num_opt = options.shape[0]

        # Padding options
        if num_opt < max_opt_len:
            pad = torch.zeros((max_opt_len - num_opt, 1024))
            options = torch.cat([options, pad], dim=0)  # shape: (max_opt_len, 1024)
        options_batch.append(options)

        label_batch.append(sample['label'])

        option_mask.append(torch.tensor([1]*num_opt + [0]*(max_opt_len - num_opt)))

    return {
        'context': torch.stack(context_batch),            # (B, num_ctx, 1024)
        'options': torch.stack(options_batch),            # (B, max_opt_len, 1024)
        'label': torch.tensor(label_batch),              # (B,)
        'option_mask': torch.stack(option_mask)           # (B, max_opt_len)
    }


In [ ]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

dataset = ComicPAPDataset(json_path="/content/drive/MyDrive/projet_bd/results/dataset_embeddings2.json")
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=8, collate_fn=pad_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=pad_collate_fn)

In [ ]:
for batch in train_loader:
    print("Context shape:", batch['context'].shape)
    print("Options shape:", batch['options'].shape)
    print("Option mask shape:", batch['option_mask'].shape)
    print("Label shape:",batch['label'].shape)
    break  # 只看第一个 batch


Context shape: torch.Size([8, 4, 1024])
Options shape: torch.Size([8, 5, 1024])
Option mask shape: torch.Size([8, 5])
Label shape: torch.Size([8])


# modèle MLP

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PanelMLPClassifier(nn.Module):
    def __init__(self, embedding_dim=1024, hidden_dim=512):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(embedding_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1)
        )#mlp对

    def forward(self, context, options, option_mask):
        context_vec = context.mean(dim=1)          # [B, D]context mean pooling
        B, O, D = options.shape

        context_expanded = context_vec.unsqueeze(1).expand(-1, O, -1)  # [B, O, D]
        concat_vec = torch.cat([context_expanded, options], dim=-1)    # [B, O, 2*D]

        logits = self.mlp(concat_vec).squeeze(-1)  # [B, O]

        # minimiser les logits sur les places padding
        logits = logits.masked_fill(option_mask == 0, -1e9)
        return logits


In [ ]:
import torch.optim as optim

model=PanelMLPClassifier()
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in dataloader:
        context = batch['context'].to(device)          # [B, C, 1024]
        options = batch['options'].to(device)          # [B, O, 1024]
        option_mask = batch['option_mask'].to(device)  # [B, O]
        labels = batch['label'].to(device)             # [B]

        optimizer.zero_grad()
        logits = model(context, options, option_mask)  # [B, O]
        loss = criterion(logits, labels)#将mlp得到的实数和正确答案放入loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Accuracy
        preds = logits.argmax(dim=1)  # [B]
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / len(dataloader)
    acc = correct / total
    return avg_loss, acc



In [ ]:
num_epochs=50
for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")

Epoch 1/50
Train Loss: 1.6031 | Train Acc: 0.2105
Epoch 2/50
Train Loss: 1.5899 | Train Acc: 0.3923
Epoch 3/50
Train Loss: 1.5771 | Train Acc: 0.4354
Epoch 4/50
Train Loss: 1.5575 | Train Acc: 0.4498
Epoch 5/50
Train Loss: 1.5285 | Train Acc: 0.4641
Epoch 6/50
Train Loss: 1.4876 | Train Acc: 0.4689
Epoch 7/50
Train Loss: 1.4349 | Train Acc: 0.4737
Epoch 8/50
Train Loss: 1.3746 | Train Acc: 0.4928
Epoch 9/50
Train Loss: 1.3108 | Train Acc: 0.4880
Epoch 10/50
Train Loss: 1.2451 | Train Acc: 0.5215
Epoch 11/50
Train Loss: 1.1775 | Train Acc: 0.5263
Epoch 12/50
Train Loss: 1.1137 | Train Acc: 0.5646
Epoch 13/50
Train Loss: 1.0509 | Train Acc: 0.5837
Epoch 14/50
Train Loss: 0.9948 | Train Acc: 0.6029
Epoch 15/50
Train Loss: 0.9385 | Train Acc: 0.6411
Epoch 16/50
Train Loss: 0.8896 | Train Acc: 0.6890
Epoch 17/50
Train Loss: 0.8414 | Train Acc: 0.7081
Epoch 18/50
Train Loss: 0.7986 | Train Acc: 0.7225
Epoch 19/50
Train Loss: 0.7590 | Train Acc: 0.7321
Epoch 20/50
Train Loss: 0.7361 | Train A

In [ ]:
## ATTENTION + MLP:
class AttnMLPClassifier(nn.Module):
    def __init__(self, input_dim=1024):
        super().__init__()
        self.attn = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )
        self.mlp = nn.Sequential(
            nn.Linear(input_dim * 2, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1)
        )

    def forward(self, context, options, option_mask):
        # context: [B, C, D], options: [B, O, D]
        attn_score = self.attn(context).softmax(dim=1)   # [B, C, 1]
        context_vec = (context * attn_score).sum(dim=1)  # [B, D]

        B, O, D = options.size()
        context_exp = context_vec.unsqueeze(1).expand(-1, O, -1)  # [B, O, D]
        concat = torch.cat([context_exp, options], dim=-1)       # [B, O, 2D]

        logits = self.mlp(concat).squeeze(-1)  # [B, O]
        logits = logits.masked_fill(option_mask == 0, -1e9)
        return logits

import torch.optim as optim
model=AttnMLPClassifier()
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in dataloader:
        context = batch['context'].to(device)          # [B, C, 1024]
        options = batch['options'].to(device)          # [B, O, 1024]
        option_mask = batch['option_mask'].to(device)  # [B, O]
        labels = batch['label'].to(device)             # [B]

        optimizer.zero_grad()
        logits = model(context, options, option_mask)  # [B, O]
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Accuracy
        preds = logits.argmax(dim=1)  # [B]
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / len(dataloader)
    acc = correct / total
    return avg_loss, acc



In [ ]:
num_epochs=40
for epoch in range(num_epochs):
  train_loss, train_acc = train_one_epoch(model, loader, optimizer, criterion, device)

  print(f"Epoch {epoch+1}/{num_epochs}")
  print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")

Epoch 1/40
Train Loss: 1.6084 | Train Acc: 0.1527
Epoch 2/40
Train Loss: 1.6008 | Train Acc: 0.2176
Epoch 3/40
Train Loss: 1.5860 | Train Acc: 0.2824
Epoch 4/40
Train Loss: 1.5949 | Train Acc: 0.2176
Epoch 5/40
Train Loss: 1.5948 | Train Acc: 0.2405
Epoch 6/40
Train Loss: 1.5936 | Train Acc: 0.2252
Epoch 7/40
Train Loss: 1.5904 | Train Acc: 0.2252
Epoch 8/40
Train Loss: 1.5990 | Train Acc: 0.2710
Epoch 9/40
Train Loss: 1.5933 | Train Acc: 0.2252
Epoch 10/40
Train Loss: 1.5769 | Train Acc: 0.2901
Epoch 11/40
Train Loss: 1.5841 | Train Acc: 0.2710
Epoch 12/40
Train Loss: 1.5772 | Train Acc: 0.3015
Epoch 13/40
Train Loss: 1.5890 | Train Acc: 0.2405
Epoch 14/40
Train Loss: 1.5738 | Train Acc: 0.2710
Epoch 15/40
Train Loss: 1.5692 | Train Acc: 0.3053
Epoch 16/40
Train Loss: 1.5365 | Train Acc: 0.3435
Epoch 17/40
Train Loss: 1.5663 | Train Acc: 0.2863
Epoch 18/40
Train Loss: 1.5470 | Train Acc: 0.3092
Epoch 19/40
Train Loss: 1.5265 | Train Acc: 0.3702
Epoch 20/40
Train Loss: 1.5201 | Train A